**we need to attach contexts to lfqa test split using pinecone and wiki snippets**





In [1]:
!pip install -qU datasets pinecone-client==3.1.0 sentence-transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 840.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 29.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

In [ ]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=1234)


In [ ]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

In [ ]:
# filter only documents with History as section_title
history = wiki_data.filter(
    lambda d: d['section_title'].startswith('History')
)

In [ ]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 250000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(wiki_data, total=total_doc_count):
    # extract the fields we need
    doc = {
        "article_title": d["article_title"],
        "section_title": d["section_title"],
        "passage_text": d["passage_text"]
    }
    # add the dict containing fields we need to docs list
    docs.append(doc)

    # stop iteration once we reach 50k
    if counter == total_doc_count:
        break

    # increase the counter on every iteration
    counter += 1

In [ ]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

In [2]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'b17df4d6-5540-42ae-8a4f-9f955b312760'

# configure client
pc = Pinecone(api_key=api_key)

In [3]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name = "test-support-docs"
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=768,
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

In [3]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch["passage_text"].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

In [4]:
from datasets import load_dataset

lfqa = load_dataset('vblagoje/lfqa')

Generating train split:   0%|          | 0/226147 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3020 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
lfqa


In [5]:
# connect to index
index_name = "test-support-docs"

index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 470000}},
 'total_vector_count': 470000}

In [6]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
    return xc

In [7]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the  tag
    context = [f" {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

In [ ]:
query = "why can restaurants charge me for adding things (i.e., add sour cream), but won't lower the subtotal should i ask for things off?"
result = query_pinecone(query, top_k=4)
result

In [ ]:
lfqa['test']

In [10]:
from tqdm.auto import tqdm  # progress bar
lfqa_support = []

for item in tqdm(lfqa['test']):
    id = item['q_id']
    question = item['title']

    best_answer_index = item['answers']['score'].index(max(item['answers']['score']))
    best_answer = item['answers']['text'][best_answer_index]
    entry = {
        'id': id,
        'question': question,
        'answer': best_answer
    }
    lfqa_support.append(entry)




  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
lfqa_support

In [17]:
for item in tqdm(lfqa['test']):
    question = item['title']  # الحصول على السؤال

    # الحصول على الإجابة ذات أعلى تقييم
    best_answer_index = item['answers']['score'].index(max(item['answers']['score']))
    best_answer = item['answers']['text'][best_answer_index]

    # حفظ السؤال والإجابة ذات أعلى تقييم في القاموس
    lfqa_support[question] = best_answer

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
lfqa_support

In [ ]:
query = lfqa['test'][0]['title']
matches = query_pinecone(query, top_k=4)
matches

In [25]:
test_with_support = []

for item in tqdm(lfqa_support):
    query = item['answer']
    q_id = item['id']
    question = item['question']
    # الحصول على أفضل 4 تطابقات
    matches = query_pinecone(query, top_k=5)

    # معالجة التطابقات وحفظها في القاموس
    if matches:
      passages = [{'doc': m['metadata']['passage_text'],'score': m['score']} for m in matches['matches']]
      test_with_support.append({'id': q_id, 'question': question, 'golden_answer': query,  'passages': passages})

  0%|          | 0/10000 [00:00<?, ?it/s]

In [27]:
test_with_support[10]

{'id': '1neeot',
 'question': "Are spider's individual eyes independent of one-another?",
 'golden_answer': 'They work in pairs, but each pair can function pretty differently from the others. See [this](_URL_0_) and for something more technical [this](_URL_1_).',
 'passages': [{'doc': 'contacts perform a logical AND function, not allowing current to pass unless all the contacts are closed, so a series of "examine if on" instructions will energize its output storage bit if all the input bits are on. Similarly, a parallel set of instructions will perform a logical OR. In an electro-mechanical relay wiring diagram, a group of contacts controlling one coil is called a "rung" of a "ladder diagram ", and this concept is also used to describe PLC logic.  Some models of PLC limit the number of series and parallel instructions in one "rung" of logic. The',
   'score': 0.456001461},
  {'doc': 'each capable of carrying the entire 20+ MW load.',
   'score': 0.431978762},
  {'doc': 'Controller (NIC

In [39]:
import json
# حفظ مجموعة البيانات الجديدة إلى ملف JSON
with open('test_with_support_v4.json', 'w') as f:
    json.dump(preprocessed_dataset, f, indent=4)  # حفظ كقائمة من القواميس

In [28]:
# helpers
import re

def cleanup_references(text):
    # URL reference where we need to remove both the link text and URL
    # ...and this letter is used by most biographers as the cornerstone of Lee's personal
    # views on slavery ([1](_URL_2_ & pg=PA173), [2](_URL_1_), [3](_URL_5_)).
    # ...and this letter is used by most biographers as the cornerstone of Lee's personal views on slavery.
    result = re.sub(r"[\(\s]*\[\d+\]\([^)]+\)[,)]*", "", text, 0, re.MULTILINE)

    # URL reference where we need to preserve link text but remove URL
    # At the outbreak of the Civil War, [Leyburn left his church](_URL_19_) and joined the South.
    # At the outbreak of the Civil War, Leyburn left his church and joined the South.
    result = re.sub(r"\[([^]]+)\]\([^)]+\)", "\\1", result, 0, re.MULTILINE)

    # lastly remove just dangling _URL_[0-9]_ URL references
    result = re.sub(r"_URL_\d_", "", result, 0, re.MULTILINE)
    return result

def clean_answer(text):
    result = cleanup_references(text)
    result = result.replace("\n", " ")
    result = re.sub(r"\s\s+", " ", result)
    result = re.sub(r"BULLET::::-", "", result)
    return result.strip()

def clean_question(text):
    result = cleanup_references(text)
    result = result.replace("\n", " ")
    result = re.sub(r"\s\s+", " ", result)
    result = result.replace("[deleted]", "")
    return result.lower().strip()

def prepare_support_docs(example, threshold):
    filtered_texts = [text["doc"] for text in example["passages"] if text["score"] > threshold]

    if len(filtered_texts) >= 3:
        context = "<P> " + " <P> ".join(filtered_texts[:3])
        return context
    else:
        return None

In [35]:
# preprocessing test split
from tqdm.auto import tqdm

threshold = 0.65 # for passages

preprocessed_dataset = []

for item in tqdm(test_with_support):
  question = clean_question(item['question'])
  answer = clean_answer(item['golden_answer'])
  support_docs = prepare_support_docs(item, threshold)

  if support_docs:
      preprocessed_item = {
          "question": question,
          "answer": answer,
          "support_docs": support_docs
      }
      preprocessed_dataset.append(preprocessed_item)


  0%|          | 0/10000 [00:00<?, ?it/s]

In [36]:
len(preprocessed_dataset)

112

In [38]:
preprocessed_dataset[1]

{'question': 'the iter fusion experiment is planned to be finished with assembly by 2021 but not start fusing until 2035. why a 14 year gap?',
 'answer': '> On its way to full deuterium-tritium operation, ITER will experiment with a succession of "non-nuclear" plasma fuels. > Spread over a period of roughly seven years, hydrogen, helium (with a variable proportion of hydrogen) and deuterium campaigns—interspersed with maintenance and upgrade periods—will provide operators with the necessary know-how to run the machine, commission its components, and control its plasma before entering nuclear operation.',
 'support_docs': "<P> reactors. In addition, progress in the development of advanced, low activation structural materials supports the promise of environmentally benign fusion reactors and research into alternate confinement concepts is yielding the promise of future improvements in confinement. Finally, supporters contend that other potential replacements to the fossil fuels have envi